In [1]:
import os
import vertexai
import functions_framework
import json

from langchain_google_community import VertexAISearchRetriever
from langchain_google_vertexai import VertexAI
from langchain.prompts import PromptTemplate

PROJECT_ID = "TODO" 
REGION = "us-central1"
DATA_STORE_LOCATION="global"
DATA_STORE_ID="TODO"

prompt_template = """Use the following pieces of context to answer the question at the end.

    {context}

    Question: {question}
    Answer:"""

@functions_framework.http
def searchSample(request):
    search_query = request.form['text']
    vertexai.init(project=PROJECT_ID, location=REGION)

    retriever = VertexAISearchRetriever(
        project_id=PROJECT_ID,
        location_id=DATA_STORE_LOCATION,
        data_store_id=DATA_STORE_ID,
        max_documents=5,
    )

    result = retriever.invoke(search_query)
    
    llm = VertexAI(
        model_name="gemini-1.5-flash-001",
        temperature=0,
        verbose=True
    )

    prompt = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    response = llm.invoke(prompt.format(context =result, question=search_query))
    
      # Formating answer to make it compatible with slack
    data = {}
    data['blocks'] = []
    data['blocks'].append({"type":"section",
                   "text": {
                          "type": "mrkdwn",
                          "text": "Answer :" + response
                      }
                  })
    return json.dumps(data),200, {'Content-Type': 'application/json'}
